In [1]:
import pyspark
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pyspark.sql import SparkSession
from pyspark.sql import functions as func
from pyspark.sql.types import IntegerType,StringType,DoubleType
from pyspark.ml.stat import Correlation
from pyspark.sql.functions import split,udf,col,regexp_replace

## Creating a spark session

In [2]:
conf = pyspark.SparkConf().setMaster('local[*]') \
        .set('spark.executor.heartbeatInterval', 10000) \
        .set('spark.network.timeout', 10000) \
        .set('spark.core.connection.ack.wait.timeout', '3600')
spark = SparkSession \
        .builder \
        .appName('parking') \
        .config(conf=conf) \
        .getOrCreate()

## Loading dataset and overview

In [3]:
df = spark.read.csv('./parking.csv',header=True,inferSchema=True)

In [4]:
pd.DataFrame(df.dtypes,columns=['column Name', 'Data type'])

,column Name,Data type
0,Summons Number,bigint
1,Plate ID,string
2,Registration State,string
3,Plate Type,string
4,Issue Date,string
5,Violation Code,int
6,Vehicle Body Type,string
7,Vehicle Make,string
8,Issuing Agency,string
9,Street Code1,int


In [5]:
df.describe().toPandas()

,summary,Summons Number,Plate ID,Registration State,Plate Type,Issue Date,Violation Code,Vehicle Body Type,Vehicle Make,Issuing Agency,...,Hydrant Violation,Double Parking Violation,Latitude,Longitude,Community Board,Community Council,Census Tract,BIN,BBL,NTA
0,count,9100278,9100276,9100278,9100278,9100278,9100278,8989410,9037103,9100278,...,1,1,1,1,1,1,1,1,1,1
1,mean,6.486507134673543E9,NaN,99.0,999.0,None,35.25493495912982,5.256613226452906,4555.626865671642,None,...,None,None,None,None,None,None,None,None,None,None
2,stddev,2.1664474212002106E9,NaN,0.0,9.85113404791972E-14,None,20.50547511932227,23.127694456636892,12905.597273858535,None,...,None,None,None,None,None,None,None,None,None,None
3,min,1001793950,!,99,999,01/01/1981,0,-,(UPS),A,...,"{""""id"""":135110655","name"""":""""Drug Maximum Out of Pocket - individu...","dataTypeName"""":""""text","fieldName"""":""""drug_maximum_out_of_pocket_indiv...","position"""":64","tableColumnId"""":16639867","width"""":700","format"""":{}","metadata"""":{}}","{""""id"""":135110656"
4,max,8006150291,],YT,WUG,12/31/2031,99,ZSR,`,X,...,"{""""id"""":135110655","name"""":""""Drug Maximum Out of Pocket - individu...","dataTypeName"""":""""text","fieldName"""":""""drug_maximum_out_of_pocket_indiv...","position"""":64","tableColumnId"""":16639867","width"""":700","format"""":{}","metadata"""":{}}","{""""id"""":135110656"


In [6]:
df.limit(2).toPandas()

,Summons Number,Plate ID,Registration State,Plate Type,Issue Date,Violation Code,Vehicle Body Type,Vehicle Make,Issuing Agency,Street Code1,...,Hydrant Violation,Double Parking Violation,Latitude,Longitude,Community Board,Community Council,Census Tract,BIN,BBL,NTA
0,1283294138,GBB9093,NY,PAS,08/04/2013,46,SUBN,AUDI,P,37250,...,None,None,None,None,None,None,None,None,None,None
1,1283294151,62416MB,NY,COM,08/04/2013,46,VAN,FORD,P,37290,...,None,None,None,None,None,None,None,None,None,None


## Pre-Processing

<font size="4">In the following columns, if we see the values of column 'Violation Time' it should actually be in the format HH:MM:AM/PM. (Ex:- 0752A) should be 07:52:AM and if we see the values of column 'Issue Date' it can be split into 3 columns 'Date', 'Month', 'Year' respectivley</font>

In [7]:
df.select('Violation Time').show(5)

+--------------+
|Violation Time|
+--------------+
|         0752A|
|         1240P|
|         1243P|
|         0232P|
|         1239P|
+--------------+
only showing top 5 rows



In [8]:
df.select('Issue Date').show(5)

+----------+
|Issue Date|
+----------+
|08/04/2013|
|08/04/2013|
|08/05/2013|
|08/05/2013|
|08/08/2013|
+----------+
only showing top 5 rows



<font size="4">In the following cells we are splitting the column 'Violation Time' into 'Time_Hour' and 'Meridiem'. Column 'Time_Hour' contains the time the violation was commited and column 'Meridiem' contains if it was during AM or PM. Column 'Issue Date' is split into 'Day' **(typo it should be Date)**, 'Year' and 'Month'.</font>

In [4]:
import datetime 
def day_finder(x):
    return datetime.datetime.strptime(x, '%m/%d/%Y').weekday()
day_udf = func.udf(lambda x: day_finder(x), IntegerType())

In [5]:
df = df.withColumnRenamed('Violation Time', 'Violation_Time')
df_new = df.withColumn('Month',func.split('Issue Date','/')[0]) \
            .withColumn('Year',func.split('Issue Date','/')[2]) \
            .withColumn('Day',day_udf(func.col('Issue Date')))  \
            .withColumn('Meridiem', \
                      func.when(func.isnan(df.Violation_Time) \
                               | func.col('Violation_Time').isNull()\
                               , func.lit(None))\
                      .otherwise(func.substring(df.Violation_Time,5,1))) \
            .withColumn('Time_Hour', \
                      func.when(func.isnan(df.Violation_Time) \
                               | func.col('Violation_Time').isNull()\
                               , func.lit(None))\
                      .otherwise(func.substring(df.Violation_Time,1,2)))

In [6]:
df_new = df_new.withColumn('Year',df_new['Year'].cast(IntegerType())) \
    .withColumn("Month",df_new["Month"].cast(DoubleType())) \
    .withColumn("Day",df_new["Day"].cast(DoubleType())) \
    .withColumn("Time_Hour",df_new["Time_Hour"].cast(DoubleType()))

In [12]:
df_new.groupBy("Year") \
  .count() \
  .withColumnRenamed('count', 'count') \
  .withColumn('percentage', (func.col('count') / df_new.count()) * 100 ) \
  .show()

+----+-------+--------------------+
|Year|  count|          percentage|
+----+-------+--------------------+
|2003|     39|4.285583363497247E-4|
|2007|     13|1.428527787832415...|
|2018|    181|0.001988950227674...|
|2044|      9|9.889807761916724E-5|
|2015|   1522| 0.01672476379293028|
|2032|      3|3.296602587305575E-5|
|2023|     31|3.406489340215760...|
|2006|     11|1.208754282012044...|
|2031|     93|0.001021946802064...|
|2013|4379109|   48.12060686497709|
|2014|4716512|  51.828218874192636|
|1973|     10|1.098867529101858...|
|2041|     39|4.285583363497247E-4|
|2019|    329|0.003615274170745...|
|2004|     77|8.461279974084309E-4|
|2030|     45|4.944903880958363E-4|
|2053|      1|1.098867529101858...|
|2012|    618|0.006791001329849484|
|2009|      9|9.889807761916724E-5|
|2016|    296|0.003252647886141...|
+----+-------+--------------------+
only showing top 20 rows



<font size="4">The 'Year' the year value range varies in large (few misinputs). This dataset's primary year being 2013-2014 in the following cell we remove all the rows that does not have a year value of either 2013 or 2014</font>

In [7]:
df_new = df_new.where(func.col('Year')<2015)
df_new = df_new.where(func.col('Year')>2012)

In [14]:
df_new.groupBy("Meridiem") \
  .count() \
  .withColumnRenamed('count', 'count') \
  .withColumn('percentage', (func.col('count') / df_new.count()) * 100 ) \
  .show()

+--------+-------+--------------------+
|Meridiem|  count|          percentage|
+--------+-------+--------------------+
|    null|   2054|0.022582295370486522|
|       A|4436275|   48.77374508018749|
|       P|4657277|  51.203507709918874|
|        |     15|1.649145231535043E-4|
+--------+-------+--------------------+



<font size="4">The 'Meridiem' column has few null and empty values, the empty values are converted into 'None' and later on dropped</font>

In [8]:
df_new = df_new.withColumn("Meridiem", \
       func.when(col("Meridiem")=="" ,None) \
          .otherwise(col("Meridiem"))) 

In [16]:
df_new.groupBy("Time_Hour") \
  .count() \
  .withColumnRenamed('count', 'count') \
  .withColumn('percentage', (func.col('count') / df_new.count()) * 100 ) \
  .show()

+---------+------+--------------------+
|Time_Hour| count|          percentage|
+---------+------+--------------------+
|      8.0|955730|  10.507583814233245|
|      0.0| 52530|   0.577530660083572|
|      7.0|518121|   5.696378510054453|
|     29.0|    10|1.099430154356695...|
|     44.0|     1|1.099430154356695...|
|     null|  2057|0.022615278275117223|
|      1.0|968418|  10.647079512218022|
|      0.6|     1|1.099430154356695...|
|      4.0|580011|   6.376815832585812|
|     85.0|     1|1.099430154356695...|
|     77.0|     1|1.099430154356695...|
|     11.0|964384|  10.602728499791274|
|      3.0|627837|   6.902629298208446|
|     53.0|     3|3.298290463070086E-5|
|     59.0|     6|6.596580926140172E-5|
|     28.0|    15|1.649145231535043E-4|
|      2.0|857338|   9.425832496758606|
|     10.0|823190|    9.05039908764888|
|     30.0|     9| 9.89487138921026E-5|
|     84.0|     2|2.198860308713390...|
+---------+------+--------------------+
only showing top 20 rows



<font size="4">The 'Time_Hour' column has garbage values. The value should be between 1-12(12 hour format), in the following cell we remove all the rows that does not have a time value between 1.0 and 12.0</font>

In [9]:
df_new = df_new.withColumn("Time_Hour", func.when((func.col("Time_Hour") <= 0.0) |  \
                                                  (func.col("Time_Hour") > 12.0),1.0) \
                           .otherwise(df_new.Time_Hour))

In [18]:
df_new.groupBy("Registration State") \
  .count() \
  .withColumnRenamed('count', 'count') \
  .withColumn('percentage', (func.col('count') / df_new.count()) * 100 ) \
  .show(100)

+------------------+-------+--------------------+
|Registration State|  count|          percentage|
+------------------+-------+--------------------+
|                SC|  19515|  0.2145537946227091|
|                AZ|  24221| 0.26629297768673516|
|                NS|    373|0.004100874475750474|
|                LA|   2497|0.027452770954286686|
|                MN|  12897|   0.141793507007383|
|                NJ| 878359|   9.656943709505926|
|                MX|    108|0.001187384566705231|
|                DC|   3727| 0.04097576185287404|
|                OR|   3482| 0.03828215797470013|
|                99|  37912| 0.41681596011971034|
|                VA|  60919|  0.6697618557325553|
|                RI|  13286| 0.14607029030783053|
|                KY|   3222|0.035423639573372724|
|                WY|    300|0.003298290463070...|
|                BC|    329|0.003617125207833528|
|                NH|   9928| 0.10915142572453272|
|                MI|  16357| 0.17983379034812466|


<font size="4">Removing Garbage value '99' from 'Registration State' column</font>

In [10]:
df_new = df_new.withColumnRenamed('Registration State', 'Registration_State')
df_new = df_new.filter(df_new.Registration_State != '99')

In [20]:
df_new.groupBy("Plate Type") \
  .count() \
  .withColumnRenamed('count', 'count') \
  .withColumn('percentage', (func.col('count') / df_new.count()) * 100 ) \
  .show(100)

+----------+-------+--------------------+
|Plate Type|  count|          percentage|
+----------+-------+--------------------+
|       CCK|     24|2.649676645606521E-4|
|       OMO|     40|4.416127742677535...|
|       LMB|     30|3.312095807008152E-4|
|       CLG|     12|1.324838322803260...|
|       SOS|     67|7.397013968984872E-4|
|       SPC|    209|0.002307426745549...|
|       SUP|     31|3.422499000575090...|
|       NYA|    122|0.001346918961516...|
|       RGL|   4281|0.047263607166006326|
|       CHC|    262|0.002892563671453786|
|       STA|    187|0.002064539719701748|
|       RGC|    267|0.002947765268237255|
|       TRC|  22351| 0.24676217794146402|
|       AMB|     86|9.494674646756703E-4|
|       COM|1878953|  20.744241176217958|
|       HAM|    137|0.001512523751867056|
|       CMH|   2347| 0.02591162953016044|
|       NYS|     26|2.870483032740398...|
|       MCD|    102|0.001126112574382...|
|       IRP|  59785|  0.6600454927399412|
|       ORG|  12721| 0.14044390253

<font size="4">Removing Garbage value '999' from 'Plate Type' column</font>

In [11]:
df_new = df_new.withColumnRenamed('Plate Type', 'Plate_Type')
df_new = df_new.filter(df_new.Plate_Type != '999')

<font size="4">Trimming extra spaces from column name(Days Parking In Effect, Community Council)</font>

In [12]:
df_new = df_new.withColumnRenamed('Days Parking In Effect    ','Days Parking In Effect')
df_new = df_new.withColumnRenamed('Community Council ','Community Council')

In [23]:
df_new.groupBy("Violation County") \
  .count() \
  .withColumnRenamed('count', 'count') \
  .withColumn('percentage', (func.col('count') / df_new.count()) * 100 ) \
  .show()

+----------------+-------+--------------------+
|Violation County|  count|          percentage|
+----------------+-------+--------------------+
|               K|1958357|  21.738373200472964|
|               Q|1807826|   20.06743217376517|
|              BX| 931991|  10.345390639950732|
|            null| 694306|   7.707013043754321|
|           BRONX|      3|3.330093522346481...|
|               R|  98074|  1.0886519737020295|
|              NY|3518187|   39.05297246367867|
|            RICH|      4|4.440124696461975E-5|
|           KINGS|      1|1.110031174115493...|
|             NYC|      2|2.220062348230987...|
|             103|      1|1.110031174115493...|
|           QUEEN|      2|2.220062348230987...|
|              RC|      2|2.220062348230987...|
+----------------+-------+--------------------+



<font size="4">The target column 'Violation County' has multiple abbrevation for a single location (Ex:- NYC and NY both indicate New York), in the following cell it is made to one). Also other parking violation datasets' (2015-2016) 'Violation County' values has been viewed for confirmation </font>

In [13]:
df_new = df_new.withColumn('Violation County', regexp_replace('Violation County', 'KINGS', 'K')) \
                .withColumn('Violation County', regexp_replace('Violation County','QUEEN', 'Q')) \
                .withColumn('Violation County', regexp_replace('Violation County', 'BRONX', 'BX')) \
                .withColumn('Violation County', regexp_replace('Violation County', 'RC', 'R'))   \
                .withColumn('Violation County', regexp_replace('Violation County', 'RICH','R'))  \
                .withColumn('Violation County', regexp_replace('Violation County', 'NYC', 'NY'))

<font size="4">Since 'Violation County' is our target column in the following cells we are dropping any null rows preset and rows that has garbage value</font>

In [14]:
df_new = df_new.dropna(how='any',subset=['Violation County'])

In [15]:
df_new = df_new.withColumnRenamed('Violation County', 'Violation_County')
df_new = df_new.filter(df_new.Violation_County != '103')

<font size="4">Replacing emtpy values in the entire dataframe with None</font>

In [16]:
df_new=df_new.select([func.when(func.col(c)=="",None).otherwise(func.col(c)).alias(c) for c in df_new.columns])

<font size="4">Finding the correlation between column 'Violation Location' and 'Violation Precint'</font>

In [31]:
df_new.stat.corr('Violation Location','Violation Precinct')

1.0

<font size="4">In the following cells uncessary columns are guessed based on column names, for null values percentage of columns, kaggle has been refered</font>

In [17]:
# Uneccessary columns
columns_to_drop = ['Summons Number','Plate ID','Issuer Code','Vehicle Expiration Date','House Number','Street Name','Date First Observed','Law Section','Sub Division','Vehicle Color','Vehicle Year','Feet From Curb','Violation Post Code','Violation Description','Violation Precinct']
# Columns that has more than 30% of missing value
columns_missing2 = ['Days Parking In Effect','From Hours In Effect','To Hours In Effect']
# columns that has more than 75% of missing values
columns_missing = ['Time First Observed','Intersecting Street','Violation Legal Code','Unregistered Vehicle?','Meter Number',"No Standing or Stopping Violation","Hydrant Violation","Double Parking Violation","Latitude","Longitude","Community Board","Community Council","Census Tract","BIN","BBL","NTA"]
# Columns that are no longer needed
columns_to_drop2 = ['Year','Violation_Time','Issue Date','Issuer Squad']
final_columns = columns_missing+columns_missing2+columns_to_drop+columns_to_drop2

In [18]:
len(final_columns)

38

In [19]:
df_new = df_new.drop(*final_columns)

<font size='4'>Replacing spaces in the column name with underscore</font>

In [20]:
df_new = df_new.select([func.col(col).alias(col.replace(' ', '_')) for col in df_new.columns])

In [34]:
df_new = df_new.na.drop('any')

In [36]:
df_new.count()

8142488

In [ ]:
df_empty = df_new.select([func.count(func.when(col(c).contains('None') | \
                            col(c).contains('NULL') | \
                            (col(c) == '' ) | \
                            col(c).isNull() | \
                            func.isnan(c), c 
                           )).alias(c)
                    for c in df_new.columns]).show()

In [46]:
len(df_new.columns)

18

## ANALYSIS

In [48]:
df_new.groupBy("Month") \
  .count() \
  .withColumnRenamed('count', 'count') \
  .withColumn('percentage', (func.col('count') / df_new.count()) * 100 ) \
  .show()

+-----+------+------------------+
|Month| count|        percentage|
+-----+------+------------------+
|  8.0|734673| 9.022709029476003|
|  7.0|126908|1.5585899543235435|
|  1.0|644437|  7.91449738703944|
|  4.0|774254| 9.508813522353364|
| 11.0|769771| 9.453756640476474|
|  3.0|814310|10.000751613020492|
|  2.0|582153| 7.149571482328251|
| 10.0|858861|10.547893960666569|
|  6.0|633118| 7.775485822023932|
|  5.0|818118|10.047518645406663|
|  9.0|728177| 8.942929974229006|
| 12.0|657708| 8.077481968656263|
+-----+------+------------------+



In [49]:
df_new.groupBy("Violation_County") \
  .count() \
  .withColumnRenamed('count', 'count') \
  .withColumn('percentage', (func.col('count') / df_new.count()) * 100 ) \
  .show()

+----------------+-------+------------------+
|Violation_County|  count|        percentage|
+----------------+-------+------------------+
|               K|1912409|23.486789295851587|
|               Q|1772184|    21.76464982202|
|              BX| 906164|11.128834331717774|
|               R|  89976|1.1050185152253218|
|              NY|3461755|42.514708035185315|
+----------------+-------+------------------+



In [22]:
df_new.groupBy("Violation_County") \
  .count() \
  .withColumnRenamed('count', 'count') \
  .withColumn('percentage', (func.col('count') / df_new.count()) * 100 ) \
  .show()

+----------------+-------+------------------+
|Violation_County|  count|        percentage|
+----------------+-------+------------------+
|               K|1958358|23.553671445937066|
|               Q|1807828|21.743208720144892|
|              BX| 931994|11.209329686188466|
|               R|  98080|1.1796331903653507|
|              NY|3518189|42.314156957364226|
+----------------+-------+------------------+

